In [24]:
from pyvi import ViTokenizer
import pandas as pd
import re
import joblib
from tensorflow.keras.models import load_model

In [25]:
def preprocessing(text):
    text = text.lower()
    text = ViTokenizer.tokenize(text)
    with open('vietnamese-stopwords-dash.txt','r', encoding='utf-8') as f:
        sw = f.readlines()
    sw = [i[:-1] for i in sw]
    text = ' '.join([word.lower() for word in text.split() if word.lower() not in sw]).strip()
    text = re.sub(r'[^a-zA-Z0123456789àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ\s]', '', text.lower())
    return text

In [26]:
#load countvectorize , load onehot
count_vectorizer = joblib.load('count_vectorizer.joblib')
onehot = joblib.load('one_hot_encoder.joblib')
tfidf = joblib.load('tfidf.joblib')

In [27]:
# Load mô hình đã được huấn luyện
model = load_model('final_model.keras')


In [31]:
import numpy as np

# Nhập đầu vào từ người dùng
user_input = input('Enter your description: ')

# Kiểm tra nếu đầu vào trống
if not user_input.strip():
    print("Error: Input cannot be empty.")
else:
    try:
        # Tiền xử lý dữ liệu đầu vào
        preprocessed_input = preprocessing(user_input)  # Tiền xử lý đầu vào
        
        # Mã hóa đầu vào bằng CountVectorizer
        input_encoded = count_vectorizer.transform([preprocessed_input]).toarray()  # Vector hóa CountVectorizer
        
        # Mã hóa đầu vào bằng TfidfTransformer
        input_tfidf = tfidf.transform(input_encoded).toarray()  # Vector hóa TF-IDF
        
        # Kết hợp các đặc trưng từ CountVectorizer và TF-IDF
        input_combined = np.concatenate((input_encoded, input_tfidf), axis=1)

        # Dự đoán thể loại
        prediction = model.predict(input_combined)  # Đưa vào mô hình
        predicted_label = onehot.inverse_transform(prediction)  # Chuyển đổi dự đoán về nhãn gốc
        print('Genre:', predicted_label)

    except Exception as e:
        print(f"An error occurred during processing: {e}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Genre: [['Kỹ Năng Sống']]
